In [1]:
# Import dependencies
import pandas as pd
import gmaps
from config import g_key
import requests

In [2]:
# Import the weatherpy_challenge csv file as a dataframe
city_data_df = pd.read_csv("weather_data/WeatherPy_Challenge.csv")
city_data_df

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Hithadhoo,MV,2020-04-17 19:28:23,-0.60,73.08,84.81,72,36,7.27,scattered clouds,0.00,0
1,1,Codrington,AU,2020-04-17 19:26:43,-38.27,141.97,55.90,76,90,12.77,light rain,0.16,0
2,2,Coquimbo,CL,2020-04-17 19:28:24,-29.95,-71.34,62.60,77,0,16.11,clear sky,0.00,0
3,3,Mataura,NZ,2020-04-17 19:28:25,-46.19,168.86,50.00,91,100,7.00,light rain,0.12,0
4,4,Bilibino,RU,2020-04-17 19:28:27,68.05,166.44,-2.24,85,12,4.76,few clouds,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,557,Viseu,PT,2020-04-17 19:32:01,40.66,-7.91,61.00,1,4,3.00,clear sky,0.00,0
558,558,Verkhneyarkeyevo,RU,2020-04-17 19:34:50,55.45,54.31,38.61,83,42,11.61,scattered clouds,0.00,0
559,559,Rio Gallegos,AR,2020-04-17 19:34:51,-51.62,-69.22,60.80,38,0,19.46,clear sky,0.00,0
560,560,Saint Stephen,GB,2020-04-17 19:34:52,50.33,-4.90,52.00,93,100,23.04,light rain,0.45,0


In [3]:
# Ask the customer questions to filter the data
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = str(input("Do you want it to be raining? (yes/no) "))
snowing = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataframe based on the responses

min_max_temp_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

if (raining == "no") & (snowing == "no"):
    preferred_cities_df = min_max_temp_cities.loc[(min_max_temp_cities["Rain inches (last 3 hrs)"] == 0.00) & (min_max_temp_cities["Snow inches (last 3 hrs)"] == 0)]
    
elif (raining == "yes") & (snowing == "no"):
    preferred_cities_df = min_max_temp_cities.loc[(min_max_temp_cities["Rain inches (last 3 hrs)"] > 0.00) & (min_max_temp_cities["Snow inches (last 3 hrs)"] == 0)]

elif (raining == "no") & (snowing == "yes"):
    preferred_cities_df = min_max_temp_cities.loc[(min_max_temp_cities["Rain inches (last 3 hrs)"] == 0.00) & (min_max_temp_cities["Snow inches (last 3 hrs)"] > 0)]
    
elif (raining == "yes") & (snowing == "yes"):
    preferred_cities_df = min_max_temp_cities.loc[(min_max_temp_cities["Rain inches (last 3 hrs)"] > 0.00) & (min_max_temp_cities["Snow inches (last 3 hrs)"] > 0)]
    
else:
    print("Please enter yes or no")

In [7]:
# Create dataframe called hotel_df to store hotel names along with city, country, max temp, current weather and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

In [9]:
# Iterate through the dataframe
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# Display the dataframe with the added hotel info
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Coquimbo,CL,62.60,clear sky,-29.95,-71.34,Hotel San Juan
6,Rikitea,PF,76.86,broken clouds,-23.12,-134.97,Pension Maro'i
7,Castro,BR,73.47,clear sky,-24.79,-50.01,CHACARA BAILLY
8,Mahajanga,MG,78.80,few clouds,-15.72,46.32,Restaurant Greedy Coco Lodge Majunga
9,Barreiro,PT,61.00,few clouds,38.66,-9.07,Residencial Principe Guest House
...,...,...,...,...,...,...,...
549,Saint Andrews,US,75.99,scattered clouds,34.05,-81.13,Baymont by Wyndham Columbia Northwest
550,Pathein,MM,74.37,clear sky,16.78,94.73,Pathein Hotel
557,Viseu,PT,61.00,clear sky,40.66,-7.91,Montebelo Viseu Congress Hotel
559,Rio Gallegos,AR,60.80,clear sky,-51.62,-69.22,Hotel Santa Cruz


In [20]:
# Create the info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} \u00b0F</dd>
</dl>
"""

# Store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Configure gmaps to use Google API key
gmaps.configure(api_key=g_key)

In [30]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
### Save the hotel dataframe as a csv file

# Create the output file
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the hotel_df to the csv
hotel_df.to_csv(output_data_file, index_label="City ID")